In [3]:

import pandas as pd
import numpy as np


df = pd.read_excel("waterquality.xlsx")

molar_mass = {
    "Ca": 40.08,
    "Mg": 24.31,
    "Na": 22.99,
    "K": 39.10,
    "F": 18.998,
    "Cl": 35.45,
    "SO4": 96.06,
    "HCO3": 61.016
}

for ion in ["Ca", "Mg", "Na", "K", "F", "Cl", "SO4", "HCO3"]:
    df[ion+"_mol"] = df[ion] / molar_mass[ion] / 1000  # mg/L → mol/L

charges = {
    "Ca": 2,
    "Mg": 2,
    "Na": 1,
    "K": 1,
    "F": -1,
    "Cl": -1,
    "SO4": -2,
    "HCO3": -1
}

def ionic_strength(row):
    I = 0
    for ion in charges.keys():
        I += row[ion+"_mol"] * charges[ion]**2
    return 0.5 * I

df["IonicStrength"] = df.apply(ionic_strength, axis=1)

def activity_coeff(z, I):
    return 10 ** (-0.51 * z**2 * (np.sqrt(I)/(1+np.sqrt(I)) - 0.2*I))

df["gamma_Ca"] = df["IonicStrength"].apply(lambda x: activity_coeff(2, x))
df["gamma_F"] = df["IonicStrength"].apply(lambda x: activity_coeff(1, x))


df["IAP_CaF2"] = df["Ca_mol"] * (df["F_mol"]**2) * df["gamma_Ca"] * (df["gamma_F"]**2)

Ksp_CaF2 = 3.9e-11
df["SI_CaF2"] = np.log10(df["IAP_CaF2"] / Ksp_CaF2)


def classify_si(si):
    if si < 0:
        return 'Undersaturated'
    elif si == 0:
        return 'Saturated'
    else:
        return 'Oversaturated'

df["Saturation_Status"] = df["SI_CaF2"].apply(classify_si)


summary = df['Saturation_Status'].value_counts(normalize=True) * 100
print("Percentage of samples in each category:")
print(summary)

df.to_excel("waterqualiy_SI_results.xlsx", index=False)
print("Results saved to 'waterqualiy_SI_results.xlsx'")

print(df[["Sl no.", "Ca", "F", "SI_CaF2", "Saturation_Status"]].head(10))


Percentage of samples in each category:
Saturation_Status
Undersaturated    86.746988
Oversaturated     13.253012
Name: proportion, dtype: float64
Results saved to 'waterqualiy_SI_results.xlsx'
   Sl no.  Ca    F   SI_CaF2 Saturation_Status
0       1  42  4.0 -0.175275    Undersaturated
1       2  48  1.4 -1.047564    Undersaturated
2       3  32  1.2 -1.370242    Undersaturated
3       4  20  2.3 -0.917040    Undersaturated
4       5  12  6.4 -0.341565    Undersaturated
5       6  20  2.0 -0.997757    Undersaturated
6       7  16  2.0 -1.075348    Undersaturated
7       8  28  2.3 -0.899848    Undersaturated
8       9  30  2.5 -0.674884    Undersaturated
9      10  29  0.4 -2.340752    Undersaturated
